In [88]:
import ee
ee.Authenticate()

Enter verification code:  4/1AeanS0bnhhgWG2rTpk5k2GbLH7uGFA3_YJp3qgok0Jc5Q-bFoil-JWTIhNc



Successfully saved authorization token.


In [156]:
import ee
import geemap
import os
import ipywidgets as widgets
import pandas as pd

In [157]:
# variables globales
PROPIEDAD = None

In [176]:
os.environ["ROADMAP"] = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
os.environ["SATELLITE"] = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
os.environ["TERRAIN"] = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}'
os.environ["HYBRID"] = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

Map = geemap.Map()
Map.add_basemap("SATELLITE")

In [177]:
vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# visualizacion de la imagen en RGB
viz_img = {
    'bands':['RED','GREEN','BLUE'],
    'min':0.0,
    'max':0.5,
    'fillColorOpacity': '0'
}

In [178]:
def cargar_lotes(gui):
    global PROPIEDAD
    PROPIEDAD = ee.FeatureCollection(ruta_cat.value)\
                .filter(ee.Filter.eq('unidad_01', cod_prop.value))
    if PROPIEDAD.size().getInfo() > 0:
        establecer_estado('Propiedad: ' + PROPIEDAD.first().get('unidad_02').getInfo())
    else:
        establecer_estado('Código de propiedad incorrecto...!!!')
    Map.centerObject(PROPIEDAD.geometry(), 14)
    Map.addLayer(PROPIEDAD.style(**vis_params_caña), {}, 'Lotes')
    Map.remove_labels()
    Map.add_labels(PROPIEDAD, 'unidad_05', font_size='9pt', font_color='black', font_family='arial', font_weight='bold')

# funcion de buscar imagenes sat, para el boton de Buscar Imagenes
def buscar_imgs(gui):
    global PROPIEDAD, DF_IMGS
    establecer_estado('Buscando imagenes...')
    coleccion = ee.ImageCollection('COPERNICUS/S2_SR')\
                .filterBounds(PROPIEDAD.geometry())\
                .filterDate(fecha_inicio.value.strftime('%Y-%m-%d'), fecha_final.value.strftime('%Y-%m-%d'))\
                .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 99)
    # extraer IDs de la coleccion
    ids = coleccion.reduceColumns(ee.Reducer.toList(), ['system:index']).get('list').getInfo()
    # extraer fechas
    fechas = get_fechas(coleccion).getInfo()
    # tabla de datos
    dic = {'FECHA':fechas, 'ID':ids}
    DF_IMGS = pd.DataFrame(dic)
    lista=[]
    contador=0
    for i in DF_IMGS.values.tolist():
        texto = str(contador) + ' - ' + i[0] + ' (' + i[1] + ')'
        lista.append((texto, contador))
        contador+=1
    lista_imgs.options=lista
    establecer_estado(str(contador) + ' imagentes encontradas...!!!')
    
# funcion para el control DROPDOWN, cuando selecciona una imagen
def cargar_img(gui):
    index = gui.new
    
    id_imagen = DF_IMGS.loc[index]['ID']

    # carga la imagen con las bandas de interes
    img = ee.Image('COPERNICUS/S2_SR/'+id_imagen)\
            .multiply(0.0001)\
            .select(['B2','B3','B4','B8'], ['BLUE','GREEN','RED','NIR'])
    Map.addLayer(img, viz_img, 'Imagen Sat')
    Map.addLayer(PROPIEDAD.style(**vis_params_caña), {}, 'Lotes')
    
# extraer fechas de ImageCollection
def get_fechas(coll):
    def extraer_fecha(img):
        return ee.Image(img).date().format('dd/MM/YYYY')
    return coll.toList(coll.size(),0).map(extraer_fecha)    

def establecer_estado(estado):
    out.clear_output()
    with out:
        print(estado)

In [179]:
# textbox catastro
ruta_cat = widgets.Text(
    value='projects/ee-bismarksr17/assets/LOTES_2024',
    placeholder='Ruta de CATASTRO',
    disabled=False
)

# textbox codigo cañero
cod_prop = widgets.BoundedIntText(
    value=0,
    min=0,
    max=99999,
    step=1,
    disabled=False
)

# boton buscar propiedad
btn_cargar_prop = widgets.Button(
    description='Cargar Propiedad',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cargar',
)
btn_cargar_prop.on_click(cargar_lotes)

# datepiker para fecha inicio
fecha_inicio = widgets.DatePicker(
    disabled=False
)

# datepiker para fecha fin
fecha_final = widgets.DatePicker(
    disabled=False
)

# boton buscar imagenes SENTINEL-2
btn_buscar_imgs = widgets.Button(
    description='Buscar Imagenes',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cargar',
)
btn_buscar_imgs.on_click(buscar_imgs)

# dropdown control, para mostrar coleccion de imagenes
lista_imgs = widgets.Dropdown(
    options=['-'],
    value='-'
)
lista_imgs.observe(cargar_img, names='index')

In [180]:
form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    width='100%'
)

form_items = [
    widgets.Box([widgets.Label(value='CATASTRO:'), ruta_cat], layout=form_item_layout),
    widgets.Box([widgets.Label(value='COD PROP:'), cod_prop], layout=form_item_layout),
    widgets.Box([btn_cargar_prop], layout=form_item_layout),
    widgets.Box([widgets.Label(value='FECHA INICIO:'), fecha_inicio], layout=form_item_layout),
    widgets.Box([widgets.Label(value='FECHA FINAL :'), fecha_final], layout=form_item_layout),
    widgets.Box([btn_buscar_imgs], layout=form_item_layout),
    widgets.Box([widgets.Label(value='IMAGENES S2:'), lista_imgs], layout=form_item_layout)
]

form = widgets.Box(form_items, layout=widgets.Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%'
))
form

Box(children=(Box(children=(Label(value='CATASTRO:'), Text(value='projects/ee-bismarksr17/assets/LOTES_2024', …

In [181]:
out = widgets.Output(layout={'border': '1px solid black'})
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [182]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…